In [1]:
pwd

"/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/Example_Systems/SmallNewEngland/ThreeZones"

In [2]:
settings_path = joinpath(pwd(), "Settings")

"/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/Example_Systems/SmallNewEngland/ThreeZones/Settings"

In [4]:
environment_path = "../../../package_activate.jl"
#include(environment_path) #Run this line to activate the Julia virtual environment for GenX; skip it, if the appropriate package versions are installed

### Set relevant directory paths
src_path = "../../../src/"

inpath = pwd()

### Load GenX
println("Loading packages")
push!(LOAD_PATH, src_path)

using DOLPHYN
using YAML

Loading packages


[ Info: Precompiling DOLPHYN [2a466132-4b02-429f-9994-5df6ddd3977f]


 --- The following files are not being included from /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src: --- 
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/GenX.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/simple_operation.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/time_domain_reduction/time_domain_reduction.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/solve_model.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/generate_model.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/configure_solver/configure_cbc.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/configure_solver/configure_clp.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/configure_solver/configure_cplex.jl
Excluding /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/configu

In [53]:
genx_settings = joinpath(settings_path, "genx_settings.yml") #Settings YAML file path for GenX
hsc_settings = joinpath(settings_path, "hsc_settings.yml") #Settings YAML file path for HSC modelgrated model
csc_settings = joinpath(settings_path, "csc_settings.yml") #Settings YAML file path for CSC model
besc_settings = joinpath(settings_path, "besc_settings.yml") #Settings YAML file path for BESC model
sf_settings = joinpath(settings_path, "syn_fuel_settings.yml") #Settings YAML file path for SF model

mysetup_genx = YAML.load(open(genx_settings)) # mysetup dictionary stores GenX-specific parameters
mysetup_hsc = YAML.load(open(hsc_settings)) # mysetup dictionary stores H2 supply chain-specific parameters
#mysetup_csc = YAML.load(open(csc_settings)) # mysetup dictionary stores CO2 supply chain-specific parameters
#mysetup_besc = YAML.load(open(besc_settings)) # mysetup dictionary stores CO2 supply chain-specific parameters
#mysetup_sf = YAML.load(open(sf_settings)) # mysetup dictionary stores CO2 supply chain-specific parameters

global_settings = joinpath(settings_path, "global_model_settings.yml") # Global settings for inte
mysetup_global = YAML.load(open(global_settings)) # mysetup dictionary stores global settings
mysetup = Dict()
mysetup = merge(mysetup_hsc, mysetup_genx, mysetup_global) #Merge dictionary - value of common keys will be overwritten by value in global_model_settings

Dict{Any, Any} with 38 entries:
  "H2PipeInteger"                           => 0
  "H2CO2Cap"                                => 0
  "NetworkExpansion"                        => 1
  "TimeDomainReductionFolder"               => "TDR_Results"
  "ModelingToGenerateAlternativeIterations" => 3
  "SystemCO2Constraint"                     => 1
  "H2G2PCommit"                             => 0
  "ModelH2Trucks"                           => 1
  "ParameterScale"                          => 1
  "H2NetworkExpansion"                      => 1
  "EnergyShareRequirement"                  => 0
  "PrintModel"                              => 0
  "TimeDomainReduction"                     => 1
  "Trans_Loss_Segments"                     => 1
  "CapacityReserveMargin"                   => 0
  "LogFile"                                 => "output.log"
  "ModelingtoGenerateAlternativeSlack"      => 0.1
  "Solver"                                  => "gurobi"
  "MethodofMorris"                          => 0
  "Mo

In [54]:
## Cluster time series myinputs if necessary and if specified by the user
TDRpath = joinpath(inpath, mysetup["TimeDomainReductionFolder"])
if mysetup["TimeDomainReduction"] == 1
    if mysetup["ModelH2"] == 1
        if (!isfile(TDRpath*"/Load_data.csv")) || (!isfile(TDRpath*"/Generators_variability.csv")) || (!isfile(TDRpath*"/Fuels_data.csv")) || (!isfile(TDRpath*"/HSC_generators_variability.csv")) || (!isfile(TDRpath*"/HSC_load_data.csv"))
            println("Clustering Time Series Data...")
            cluster_myinputs(inpath, settings_path, mysetup)
        else
            println("Time Series Data Already Clustered.")
        end
    else
        if (!isfile(TDRpath*"/Load_data.csv")) || (!isfile(TDRpath*"/Generators_variability.csv")) || (!isfile(TDRpath*"/Fuels_data.csv"))
            println("Clustering Time Series Data...")
            cluster_myinputs(inpath, settings_path, mysetup)
        else
            println("Time Series Data Already Clustered.")
        end
    end
end

Time Series Data Already Clustered.


In [55]:
# ### Configure solver
println("Configuring Solver")
OPTIMIZER = configure_solver(mysetup["Solver"], settings_path)

Configuring Solver


MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("OptimalityTol") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("FeasibilityTol") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("Presolve") => 1, MathOptInterface.RawOptimizerAttribute("AggFill") => 10, MathOptInterface.RawOptimizerAttribute("PreDual") => 0, MathOptInterface.RawOptimizerAttribute("TimeLimit") => 110000, MathOptInterface.RawOptimizerAttribute("MIPGap") => 0.0001, MathOptInterface.RawOptimizerAttribute("Method") => 4, MathOptInterface.RawOptimizerAttribute("BarConvTol") => 1.0e-8, MathOptInterface.RawOptimizerAttribute("NumericFocus") => 0, MathOptInterface.RawOptimizerAttribute("Crossover") => -1, MathOptInterface.RawOptimizerAttribute("LogFile") => "gurobi.log"])

In [58]:
# #### Running a case

# ### Load myinputs
# println("Loading myinputs")
 myinputs = Dict() # myinputs dictionary will store read-in data and computed parameters
 myinputs = load_inputs(mysetup, inpath)

# ### Load H2 myinputs if modeling the hydrogen supply chain
if mysetup["ModelH2"] == 1
    myinputs = load_h2_inputs(myinputs, mysetup, inpath)
end


Reading Input CSV Files
Network.csv Successfully Read!
Load_data.csv Successfully Read!
Fuels_data.csv Successfully Read!
Generators_data.csv Successfully Read!
Generators_variability.csv Successfully Read!
Validating time basis
CO2_cap.csv Successfully Read!
CSV Files Successfully Read In From /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/Example_Systems/SmallNewEngland/ThreeZones
Reading H2 Input CSV Files
HSC_generation.csv Successfully Read!
HSC_load_data.csv Successfully Read!
HSC_generators_variability.csv Successfully Read!
HSC_pipelines.csv Successfully Read!
HSC_zone_truck_distances_miles.csv Successfully Read!
HSC_trucks.csv Successfully Read!
HSC_g2p_variability.csv Successfully Read!
Period_map.csv Successfully Read!
HSC Input CSV Files Successfully Read In From /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/Example_Systems/SmallNewEngland/ThreeZones/


[ Info: 2023-08-28 17:31:18 Reading H2 Input CSV Files
[ Info: 2023-08-28 17:31:18 HSC_load_data.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC_generators_variability.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC_pipelines.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC_zone_truck_distances_miles.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC_trucks.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC_g2p_variability.csv Successfully Read!
[ Info: 2023-08-28 17:31:18 HSC Input CSV Files Successfully Read In From /Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/Example_Systems/SmallNewEngland/ThreeZones/


Dict{Any, Any} with 127 entries:
  "Z"                       => 3
  "H2_Voll"                 => [50000000]
  "LOSS_LINES"              => [1, 2]
  "RET_CAP_TRUCK"           => Int64[]
  "H2_G2P_RET_CAP"          => [1, 2, 3]
  "RET_CAP_CHARGE"          => Int64[]
  "pC_D_Curtail"            => [50.0, 45.0, 27.5, 10.0]
  "dfGen"                   => 10×66 DataFrame…
  "pC_H2_D_Curtail"         => [50.0]
  "pCAPEX_Comp_H2_Pipe"     => [4.35853e6, 5.78405e6, 5.78405e6]
  "H2_G2P_NAME"             => InlineStrings.String7["G2P_1", "G2P_2", "G2P_3"]
  "H2_G2P_COMMIT"           => Int64[]
  "C_G2P_Start"             => [500.0, 500.0, 500.0]
  "pTrans_Max_Possible"     => [5.9, 4.0]
  "pNet_Map"                => [1.0 -1.0 0.0; 1.0 0.0 -1.0]
  "omega"                   => [52.1429, 52.1429, 52.1429, 52.1429, 52.1429, 52…
  "pMax_Line_Reinforcement" => [2.95, 2.0]
  "RET_CAP_ENERGY"          => [8, 9, 10]
  "RESOURCES"               => InlineStrings.String31["MA_natural_gas_combined_…
  "COMM

In [23]:
using JuMP
using DataFrames

In [69]:
#Core GenX Features

include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/discharge/discharge.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/discharge/investment_discharge.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/non_served_energy.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/ucommit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/reserves.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/transmission.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/core/emissions.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/curtailable_variable_renewable/curtailable_variable_renewable.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/flexible_demand/flexible_demand.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/hydro/hydro_res.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/hydro/hydro_inter_period_linkage.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/must_run/must_run.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/storage.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/utility.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/investment_energy.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/storage_all.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/long_duration_storage.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/investment_charge.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/storage_asymmetric.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/storage/storage_symmetric.jl")

include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/thermal/thermal.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/thermal/thermal_commit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/resources/thermal/thermal_no_commit.jl")

include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/policies/co2_cap.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/policies/energy_share_requirement.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/policies/cap_reserve_margin.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/GenX/src/model/policies/minimum_capacity_requirement.jl")

#Core HSC Modelling Features
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/core/h2_investment.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/core/h2_outputs.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/core/h2_non_served.jl")

include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/flexible_demand/h2_flexible_demand.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/core/emissions_hsc.jl")

# H2 production
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/generation/h2_production_commit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/generation/h2_production_no_commit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/generation/h2_production_all.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/generation/h2_production.jl")

# H2 pipelines
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/transmission/h2_pipeline.jl")

# H2 trucks
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/truck/h2_truck_investment.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/truck/h2_truck.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/truck/h2_truck_all.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/truck/h2_long_duration_truck.jl")

# H2 storage
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/storage/h2_storage_investment_energy.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/storage/h2_storage_investment_charge.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/storage/h2_storage.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/storage/h2_storage_all.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/storage/h2_long_duration_storage.jl")

# H2 G2P
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p_investment.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p_discharge.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p_all.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p_commit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p_no_commit.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/g2p/h2_g2p.jl")

# Policies
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/HSC/model/policies/co2_cap_hsc.jl")
include("/Users/lesarmstrong/Documents/GitHub/DOLPHYN-dev/src/multisector/genx_hsc/co2_cap_power_hsc.jl")

ModelScalingFactor = 1

1

In [66]:
println(mysetup)

Dict{Any, Any}("H2PipeInteger" => 0, "H2CO2Cap" => 0, "NetworkExpansion" => 1, "TimeDomainReductionFolder" => "TDR_Results", "ModelingToGenerateAlternativeIterations" => 3, "SystemCO2Constraint" => 1, "H2G2PCommit" => 0, "ModelH2Trucks" => 1, "ParameterScale" => 1, "H2NetworkExpansion" => 1, "EnergyShareRequirement" => 0, "PrintModel" => 0, "TimeDomainReduction" => 1, "Trans_Loss_Segments" => 1, "CapacityReserveMargin" => 0, "LogFile" => "output.log", "ModelingtoGenerateAlternativeSlack" => 0.1, "Solver" => "gurobi", "MethodofMorris" => 0, "ModelH2G2P" => 1, "ModelH2Liquid" => 0, "Reserves" => 0, "StorageLosses" => 1, "OperationWrapping" => 1, "IncludeLossesInESR" => 0, "MultiStage" => 0, "OverwriteResults" => 0, "Log" => true, "UCommit" => 2, "H2GenCommit" => 0, "ModelingToGenerateAlternatives" => 0, "MaxCapReq" => 0, "MinCapReq" => 0, "CO2Cap" => 2, "ModelH2" => 1, "H2TrucksMaxDistance" => 500, "WriteShadowPrices" => 1, "Zones" => [1, 2, 3], "ModelH2Pipelines" => 1)


In [72]:

T = myinputs["T"]     # Number of time steps (hours)
Z = myinputs["Z"]     # Number of zones - assumed to be same for power and hydrogen system

## Start pre-solve timer
presolver_start_time = time()

# Generate Energy Portfolio (EP) Model
EP = Model(OPTIMIZER)

# Introduce dummy variable fixed to zero to ensure that expressions like eTotalCap,
# eTotalCapCharge, eTotalCapEnergy and eAvail_Trans_Cap all have a JuMP variable
@variable(EP, vZERO == 0);

# Initialize Power Balance Expression
# Expression for "baseline" power balance constraint
@expression(EP, ePowerBalance[t=1:T, z=1:Z], 0)

# Initialize Hydrogen Balance Expression
# Expression for "baseline" H2 balance constraint
@expression(EP, eH2Balance[t=1:T, z=1:Z], 0)



# Initialize Objective Function Expression
@expression(EP, eObj, 0)

# Power supply by z and timestep - used in emissions constraints
@expression(EP, eGenerationByZone[z=1:Z, t=1:T], 0)
@expression(EP, eTransmissionByZone[z=1:Z, t=1:T], 0)
@expression(EP, eDemandByZone[t=1:T, z=1:Z], myinputs["pD"][t, z])
# Additional demand by z and timestep - used to record power consumption in other sectors like hydrogen and carbon
@expression(EP, eAdditionalDemandByZone[t=1:T, z=1:Z], 0)    

##### Power System related modules ############
# Infrastructure
discharge!(EP, myinputs, mysetup)

non_served_energy!(EP, myinputs, mysetup)

investment_discharge!(EP, myinputs, mysetup)

if mysetup["UCommit"] > 0
    ucommit!(EP, myinputs, mysetup)
end

# Emissions of various power sector resources
emissions!(EP, myinputs)

if mysetup["Reserves"] > 0
    reserves!(EP, myinputs, mysetup)
end

if Z > 1
    transmission!(EP, myinputs, mysetup)
end

# Technologies
# Model constraints, variables, expression related to dispatchable renewable resources

if !isempty(myinputs["VRE"])
    curtailable_variable_renewable!(EP, myinputs, mysetup)
end

# Model constraints, variables, expression related to non-dispatchable renewable resources
if !isempty(myinputs["MUST_RUN"])
    must_run!(EP, myinputs, mysetup)
end

# Model constraints, variables, expression related to energy storage modeling
if !isempty(myinputs["STOR_ALL"])
    storage!(EP, myinputs, mysetup)
end

# Model constraints, variables, expression related to reservoir hydropower resources
if !isempty(myinputs["HYDRO_RES"])
    hydro_res!(EP, myinputs, mysetup)
end

# Model constraints, variables, expression related to reservoir hydropower resources with long duration storage
if mysetup["OperationWrapping"] == 1 && !isempty(myinputs["STOR_HYDRO_LONG_DURATION"])
    hydro_inter_period_linkage!(EP, myinputs)
end

# Model constraints, variables, expression related to demand flexibility resources
if !isempty(myinputs["FLEX"])
    flexible_demand!(EP, myinputs, mysetup)
end
# Model constraints, variables, expression related to thermal resource technologies
if !isempty(myinputs["THERM_ALL"])
    thermal!(EP, myinputs, mysetup)
end

# Model constraints, variables, expression related to retrofit technologies
if !isempty(myinputs["RETRO"])
    EP = retrofit(EP, myinputs)
end

###### START OF H2 INFRASTRUCTURE MODEL --- SHOULD BE A SEPARATE FILE?? ###############
if mysetup["ModelH2"] == 1
    @expression(EP, eHGenerationByZone[z=1:Z, t=1:T], 0)
    @expression(EP, eHTransmissionByZone[t=1:T, z=1:Z], 0)
    @expression(EP, eHDemandByZone[t=1:T, z=1:Z], myinputs["H2_D"][t, z])
    # Net Power consumption by HSC supply chain by z and timestep - used in emissions constraints
    @expression(EP, eH2NetpowerConsumptionByAll[t=1:T,z=1:Z], 0)    

    # Infrastructure
    EP = h2_outputs(EP, myinputs, mysetup)

    # Investment cost of various hydrogen generation sources
    EP = h2_investment(EP, myinputs, mysetup)

    if !isempty(myinputs["H2_GEN"])
        #model H2 generation
        EP = h2_production(EP, myinputs, mysetup)
    end

    # Direct emissions of various hydrogen sector resources
    EP = emissions_hsc(EP, myinputs,mysetup)

    # Model H2 non-served
    EP = h2_non_served(EP, myinputs,mysetup)

    # Model hydrogen storage technologies
    if !isempty(myinputs["H2_STOR_ALL"])
        EP = h2_storage(EP,myinputs,mysetup)
    end

    if !isempty(myinputs["H2_FLEX"])
        #model H2 flexible demand resources
        EP = h2_flexible_demand(EP, myinputs, mysetup)
    end

    if mysetup["ModelH2Pipelines"] == 1
        # model hydrogen transmission via pipelines
        EP = h2_pipeline(EP, myinputs, mysetup)
    end

    if mysetup["ModelH2Trucks"] == 1
        # model hydrogen transmission via trucks
        EP = h2_truck(EP, myinputs, mysetup)
    end

    if mysetup["ModelH2G2P"] == 1
        #model H2 Gas to Power
        EP = h2_g2p(EP, myinputs, mysetup)
    else
        # Quick fix to ensure that the H2_G2P variable is defined even if the function is not run
        # FIX ME: This needs to be handled better in co2_cap_hsc and co2_cap_power_hsc
        @expression(EP, eH2DemandByZoneG2P[z=1:Z, t=1:T], # the unit is tonne/hour
            0.0
        )
    end

    EP[:eAdditionalDemandByZone] += EP[:eH2NetpowerConsumptionByAll]
end


################  Policies #####################3
# CO2 emissions limits for the power sector only
if mysetup["ModelH2"] ==0
    co2_cap!(EP, myinputs, mysetup)
elseif mysetup["ModelH2"]==1
    EP = co2_cap_power_hsc(EP, myinputs, mysetup)
end

# Endogenous Retirements
if mysetup["MultiStage"] > 0
    endogenous_retirement!(EP, myinputs, mysetup)
end

# Energy Share Requirement
if mysetup["EnergyShareRequirement"] >= 1
    energy_share_requirement!(EP, myinputs, mysetup)
end

#Capacity Reserve Margin
if mysetup["CapacityReserveMargin"] > 0
    cap_reserve_margin!(EP, myinputs, mysetup)
end

if (mysetup["MinCapReq"] == 1)
    minimum_capacity_requirement!(EP, myinputs, mysetup)
end

if (mysetup["MaxCapReq"] == 1)
    EP = maximum_capacity_requirement(EP, myinputs)
end


## Define the objective function
@objective(EP,Min,EP[:eObj])

## Power balance constraints
# demand = generation + storage discharge - storage charge - demand deferral + deferred demand satisfaction - demand curtailment (NSE)
#          + incoming power flows - outgoing power flows - flow losses - charge of heat storage + generation from NACC
@constraint(EP, cPowerBalance[t=1:T, z=1:Z], EP[:ePowerBalance][t,z] == myinputs["pD"][t,z])

if mysetup["ModelH2"] == 1
    ###Hydrogen Balance constraints
    @constraint(EP, cH2Balance[t=1:T, z=1:Z], EP[:eH2Balance][t,z] == myinputs["H2_D"][t,z])
end

if mysetup["ModelH2Liquid"] == 1
    ###Hydrogen Liquid Balance constraints
    @constraint(EP, cH2LiqBalance[t=1:T, z=1:Z], EP[:eH2LiqBalance][t,z] == myinputs["H2_D_L"][t,z])
end

## Record pre-solver time
presolver_time = time() - presolver_start_time
    #### Question - What do we do with this time now that we've split this function into 2?
if mysetup["PrintModel"] == 1
    if modeloutput === nothing
        filepath = joinpath(pwd(), "YourModel.lp")
        JuMP.write_to_file(EP, filepath)
    else
        filepath = joinpath(modeloutput, "YourModel.lp")
        JuMP.write_to_file(EP, filepath)
    end
    print_and_log("Model Printed")
end




Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter Presolve to value 1
Set parameter AggFill to value 10
Set parameter PreDual to value 0
Set parameter TimeLimit to value 110000
Set parameter Method to value 4
Set parameter LogFile to value "gurobi.log"
Discharge Module
Non-served Energy Module
Investment Discharge Module
Unit Commitment Module
Emissions Module (for CO2 Policy modularization
Transmission Module
Dispatchable Resources Module
Storage Resources Module
Storage Investment Module
Storage Core Resources Module
Storage Resources with Symmetric Charge/Discharge Capacity Module
Thermal (Unit Commitment) Resources Module
Hydrogen Generation and Storage Discharge Module
Hydrogen Investment Discharge Module
Hydrogen Production Module
H2 Production Core Module
H2 Production (Unit Commitment) Module
H2 Production (No Unit Commitment) Module
H2 Emissions Module for CO2 Policy modularization
Hydrogen Non-served Energy Module
Hydroge

[ Info: 2023-08-28 18:01:17 Hydrogen Generation and Storage Discharge Module
[ Info: 2023-08-28 18:01:17 Hydrogen Investment Discharge Module
[ Info: 2023-08-28 18:01:17 Hydrogen Production Module
[ Info: 2023-08-28 18:01:17 H2 Production Core Module
[ Info: 2023-08-28 18:01:17 H2 Production (Unit Commitment) Module
[ Info: 2023-08-28 18:01:17 H2 Production (No Unit Commitment) Module
[ Info: 2023-08-28 18:01:17 H2 Emissions Module for CO2 Policy modularization
[ Info: 2023-08-28 18:01:17 Hydrogen Non-served Energy Module
[ Info: 2023-08-28 18:01:17 Hydrogen Storage Module
[ Info: 2023-08-28 18:01:17 H2 Storage Energy Investment Module
[ Info: 2023-08-28 18:01:17 H2 Storage Charging Investment Module
[ Info: 2023-08-28 18:01:17 H2 Storage Core Resources Module


H2 Flexible Demand Resources Module
Hydrogen Pipeline Module
Hydrogen Truck Module
H2 Truck Investment Module
H2 Truck Core Resources Module


[ Info: 2023-08-28 18:01:17 H2 Flexible Demand Resources Module
[ Info: 2023-08-28 18:01:17 Hydrogen Pipeline Module
[ Info: 2023-08-28 18:01:18 Hydrogen Truck Module
[ Info: 2023-08-28 18:01:18 H2 Truck Investment Module
[ Info: 2023-08-28 18:01:18 H2 Truck Core Resources Module


H2 g2p demand module
C02 Policies Module for power and hydrogen system combined
C02 Policies Module


[ Info: 2023-08-28 18:01:18 H2 g2p demand module
[ Info: 2023-08-28 18:01:18 C02 Policies Module for power and hydrogen system combined


In [79]:
EP

A JuMP Model
Minimization problem with:
Variables: 55518
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 31080 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 27336 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 24232 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 53500 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: OPEX_Truck, OPEX_Truck_Compression, Truck_carbon_emission, cCO2Emissions_systemwide, cH2Balance, cH2G2PMaxCap, cH2G2PMaxRetCommit, cH2G2PMaxRetNoCommit, cH2G2PMinCap, cH2GenMaxCap, cH2GenMaxRetCommit, cH2GenMaxRetNoCommit, cH2GenMinCap, cH2NSEPerSeg, cH2SoCBalInterior, cH2SoCBalStart, cH2TruckChangeEmptyAvailInterior, cH2TruckChangeEmptyAvailStart, cH2TruckChangeEmptyTravelInterior, cH2TruckChangeEmptyTravelStart, cH2TruckChangeFullAvailInt

# Playing around with expressions

In [75]:
print_and_log("Hydrogen Pipeline Module")

Hydrogen Pipeline Module


[ Info: 2023-08-28 18:02:59 Hydrogen Pipeline Module


In [76]:
inputs = myinputs

Dict{Any, Any} with 127 entries:
  "Z"                       => 3
  "H2_Voll"                 => [50000000]
  "LOSS_LINES"              => [1, 2]
  "RET_CAP_TRUCK"           => Int64[]
  "H2_G2P_RET_CAP"          => [1, 2, 3]
  "RET_CAP_CHARGE"          => Int64[]
  "pC_D_Curtail"            => [50.0, 45.0, 27.5, 10.0]
  "dfGen"                   => 10×66 DataFrame…
  "pC_H2_D_Curtail"         => [50.0]
  "pCAPEX_Comp_H2_Pipe"     => [4.35853e6, 5.78405e6, 5.78405e6]
  "H2_G2P_NAME"             => InlineStrings.String7["G2P_1", "G2P_2", "G2P_3"]
  "H2_G2P_COMMIT"           => Int64[]
  "C_G2P_Start"             => [500.0, 500.0, 500.0]
  "pTrans_Max_Possible"     => [5.9, 4.0]
  "pNet_Map"                => [1.0 -1.0 0.0; 1.0 0.0 -1.0]
  "omega"                   => [52.1429, 52.1429, 52.1429, 52.1429, 52.1429, 52…
  "pMax_Line_Reinforcement" => [2.95, 2.0]
  "RET_CAP_ENERGY"          => [8, 9, 10]
  "RESOURCES"               => InlineStrings.String31["MA_natural_gas_combined_…
  "COMM

In [77]:
T = inputs["T"] # Model operating time steps
Z = inputs["Z"]  # Model demand zones - assumed to be same for H2 and electricity

INTERIOR_SUBPERIODS = inputs["INTERIOR_SUBPERIODS"]
START_SUBPERIODS = inputs["START_SUBPERIODS"]
hours_per_subperiod = inputs["hours_per_subperiod"]

H2_P = inputs["H2_P"] # Number of Hydrogen Pipelines
H2_Pipe_Map = inputs["H2_Pipe_Map"] 


Row,pipe_no,zone_str,d,Zone
,Int64,String,Int64,Float64
1,1,z1,1,1.0
2,2,z1,1,1.0
3,1,z2,-1,2.0
4,3,z2,1,2.0
5,2,z3,-1,3.0
6,3,z3,-1,3.0


In [82]:
### Variables ###
unregister(EP, :vH2NPipe)
unregister(EP, :vH2PipeLevel)
unregister(EP, :vH2PipeFlow_pos)
unregister(EP, :vH2PipeFlow_neg)

@variable(EP, vH2NPipe[p = 1:H2_P] >= 0) # Number of Pipes
@variable(EP, vH2PipeLevel[p = 1:H2_P, t = 1:T] >= 0) # Storage in the pipe
@variable(EP, vH2PipeFlow_pos[p = 1:H2_P, t = 1:T, d = [1, -1]] >= 0) # positive pipeflow

# Negative pipeline flow

@variable(EP, vH2PipeFlow_neg[p = 1:H2_P, t = 1:T, d = [1, -1]] >= 0) # negative pipeflow

3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, Base.OneTo(168)
    Dimension 3, [1, -1]
And data, a 3×168×2 Array{VariableRef, 3}:
[:, :, 1] =
 vH2PipeFlow_neg[1,1,1]  vH2PipeFlow_neg[1,2,1]  …  vH2PipeFlow_neg[1,168,1]
 vH2PipeFlow_neg[2,1,1]  vH2PipeFlow_neg[2,2,1]     vH2PipeFlow_neg[2,168,1]
 vH2PipeFlow_neg[3,1,1]  vH2PipeFlow_neg[3,2,1]     vH2PipeFlow_neg[3,168,1]

[:, :, -1] =
 vH2PipeFlow_neg[1,1,-1]  …  vH2PipeFlow_neg[1,168,-1]
 vH2PipeFlow_neg[2,1,-1]     vH2PipeFlow_neg[2,168,-1]
 vH2PipeFlow_neg[3,1,-1]     vH2PipeFlow_neg[3,168,-1]

In [83]:
H2_P = myinputs["H2_P"] # Number of H2 Pipelines

3

For H2_Pipe_Map, can read as looking at Zone column and matching the rows with the same pipe_no together
The Zone with the + d values is the source and the Zone with the - d value is the sink in the pipe_no pairings

In [92]:
H2_Pipe_Map

Row,pipe_no,zone_str,d,Zone
,Int64,String,Int64,Float64
1,1,z1,1,1.0
2,2,z1,1,1.0
3,1,z2,-1,2.0
4,3,z2,1,2.0
5,2,z3,-1,3.0
6,3,z3,-1,3.0


In [84]:
Source_H2_Pipe_Map = H2_Pipe_Map[H2_Pipe_Map.d .== 1, :]

Row,pipe_no,zone_str,d,Zone
,Int64,String,Int64,Float64
1,1,z1,1,1.0
2,2,z1,1,1.0
3,3,z2,1,2.0


In [85]:
Sink_H2_Pipe_Map = H2_Pipe_Map[H2_Pipe_Map.d .== -1, :]

Row,pipe_no,zone_str,d,Zone
,Int64,String,Int64,Float64
1,1,z2,-1,2.0
2,2,z3,-1,3.0
3,3,z3,-1,3.0


In [86]:
unregister(EP, :vH2NPipe)
@variable(EP, vH2NPipe[p=1:H2_P] >= 0)

3-element Vector{VariableRef}:
 vH2NPipe[1]
 vH2NPipe[2]
 vH2NPipe[3]

In [89]:
unregister(EP, :vH2PipeLevel)
@variable(EP, vH2PipeLevel[p=1:H2_P, t = 1:T] >= 0)

3×168 Matrix{VariableRef}:
 vH2PipeLevel[1,1]  vH2PipeLevel[1,2]  …  vH2PipeLevel[1,168]
 vH2PipeLevel[2,1]  vH2PipeLevel[2,2]     vH2PipeLevel[2,168]
 vH2PipeLevel[3,1]  vH2PipeLevel[3,2]     vH2PipeLevel[3,168]

In [122]:
unregister(EP, :vH2PipeFlow_pos)
@variable(EP, vH2PipeFlow_pos[p=1:H2_P, t=1:T] >= 0)

3×168 Matrix{VariableRef}:
 vH2PipeFlow_pos[1,1]  vH2PipeFlow_pos[1,2]  …  vH2PipeFlow_pos[1,168]
 vH2PipeFlow_pos[2,1]  vH2PipeFlow_pos[2,2]     vH2PipeFlow_pos[2,168]
 vH2PipeFlow_pos[3,1]  vH2PipeFlow_pos[3,2]     vH2PipeFlow_pos[3,168]

In [123]:
## THIS VALUE IS BEING REDEFINED

unregister(EP, :vH2PipeFlow_neg)
#@variable(EP, vH2PipeFlow_neg[p=1:H2_P, t = 1:T, d = [-1]] >= 0)
@variable(EP, vH2PipeFlow_neg[p=1:H2_P, t=1:T] >= 0)
#@variable(EP, vH2PipeFlow_neg[p=1:H2_P, t = 1:T] >= 0)

3×168 Matrix{VariableRef}:
 vH2PipeFlow_neg[1,1]  vH2PipeFlow_neg[1,2]  …  vH2PipeFlow_neg[1,168]
 vH2PipeFlow_neg[2,1]  vH2PipeFlow_neg[2,2]     vH2PipeFlow_neg[2,168]
 vH2PipeFlow_neg[3,1]  vH2PipeFlow_neg[3,2]     vH2PipeFlow_neg[3,168]

In [126]:
unregister(EP, :vH2PipeFlow_pos)
#@variable(EP, vH2PipeFlow_pos[p=1:H2_P, t=1:T] >= 0)
unregister(EP, :vH2PipeFlow_unidirectional)
@variable(EP, vH2PipeFlow_unidirectional[p=1:H2_P, t=1:T] >= 0)

3×168 Matrix{VariableRef}:
 vH2PipeFlow_unidirectional[1,1]  …  vH2PipeFlow_unidirectional[1,168]
 vH2PipeFlow_unidirectional[2,1]     vH2PipeFlow_unidirectional[2,168]
 vH2PipeFlow_unidirectional[3,1]     vH2PipeFlow_unidirectional[3,168]

In [124]:
### Expressions ###
# Calculate the number of new pipes
unregister(EP, :eH2NPipeNew)
@expression(EP, eH2NPipeNew[p = 1:H2_P], vH2NPipe[p] - inputs["pH2_Pipe_No_Curr"][p])

3-element Vector{AffExpr}:
 vH2NPipe[1]
 vH2NPipe[2]
 vH2NPipe[3]

In [125]:
unregister(EP, :eH2PipeFlow_net)

# Calculate net flow at each pipe-zone interfrace
@expression(
    EP,
    eH2PipeFlow_net[p = 1:H2_P, t = 1:T],
    vH2PipeFlow_pos[p, t] - vH2PipeFlow_neg[p, t]
)

3×168 Matrix{AffExpr}:
 vH2PipeFlow_pos[1,1] - vH2PipeFlow_neg[1,1]  …  vH2PipeFlow_pos[1,168] - vH2PipeFlow_neg[1,168]
 vH2PipeFlow_pos[2,1] - vH2PipeFlow_neg[2,1]     vH2PipeFlow_pos[2,168] - vH2PipeFlow_neg[2,168]
 vH2PipeFlow_pos[3,1] - vH2PipeFlow_neg[3,1]     vH2PipeFlow_pos[3,168] - vH2PipeFlow_neg[3,168]

In [109]:
setup = mysetup
unregister(EP, :eCH2Pipe)

# Capital cost of booster compressors located along each pipeline - more booster compressors needed for longer pipelines than shorter pipelines
# YS Formula doesn't make sense to me
#  ParameterScale = 1 --> objective function is in million $
#  ParameterScale = 0 --> objective function is in $

if setup["ParameterScale"] == 1
    @expression(
        EP,
        eCH2Pipe,
        sum(
            eH2NPipeNew[p] * inputs["pCAPEX_H2_Pipe"][p] / (ModelScalingFactor)^2 for
            p = 1:H2_P
        )
    )
else
    @expression(
        EP,
        eCH2Pipe,
        sum(eH2NPipeNew[p] * inputs["pCAPEX_H2_Pipe"][p] for p = 1:H2_P)
    )
end

3.1337158740016002e7 vH2NPipe[1] + 5.004906754049e7 vH2NPipe[2] + 5.004906754049e7 vH2NPipe[3]

In [108]:
EP[:eObj] += eCH2Pipe

1.0490569285714284 vP[1,1] + 0.8618380000000001 vP[3,1] + 0.005214285714285715 vP[4,1] + 1.4322808571428571 vP[6,1] + 0.005214285714285715 vP[7,1] + 0.007821428571428571 vP[8,1] + 0.007821428571428571 vP[9,1] + 0.007821428571428571 vP[10,1] + 1.0490569285714284 vP[1,2] + 0.8618380000000001 vP[3,2] + 0.005214285714285715 vP[4,2] + 1.4322808571428571 vP[6,2] + 0.005214285714285715 vP[7,2] + 0.007821428571428571 vP[8,2] + 0.007821428571428571 vP[9,2] + 0.007821428571428571 vP[10,2] + 1.0490569285714284 vP[1,3] + 0.8618380000000001 vP[3,3] + 0.005214285714285715 vP[4,3] + 1.4322808571428571 vP[6,3] + 0.005214285714285715 vP[7,3] + 0.007821428571428571 vP[8,3] + 0.007821428571428571 vP[9,3] + 0.007821428571428571 vP[10,3] + 1.0490569285714284 vP[1,4] + 0.8618380000000001 vP[3,4] + 0.005214285714285715 vP[4,4] + 1.4322808571428571 vP[6,4] + 0.005214285714285715 vP[7,4] + 0.007821428571428571 vP[8,4] + 0.007821428571428571 vP[9,4] + 0.007821428571428571 vP[10,4] + 1.0490569285714284 vP[1,5] +

In [110]:
## End Objective Function Expressions ##

## Balance Expressions ##
# H2 Power Consumption balance

unregister(EP, :ePowerBalanceH2PipeCompression)

if setup["ParameterScale"] == 1 # IF ParameterScale = 1, power system operation/capacity modeled in GW rather than MW 
    @expression(
        EP,
        ePowerBalanceH2PipeCompression[t = 1:T, z = 1:Z],
        sum(
            vH2PipeFlow_neg[
                p, t, H2_Pipe_Map[(H2_Pipe_Map[!, :Zone].==z).&(H2_Pipe_Map[!, :pipe_no].==p), :,][!,:d][1]
            ] * inputs["pComp_MWh_per_tonne_Pipe"][p] for p in H2_Pipe_Map[H2_Pipe_Map[!, :Zone].==z, :][!, :pipe_no]
        ) / ModelScalingFactor
    )
else # IF ParameterScale = 0, power system operation/capacity modeled in MW so no scaling of H2 related power consumption
    @expression(
        EP,
        ePowerBalanceH2PipeCompression[t = 1:T, z = 1:Z],
        sum(
            vH2PipeFlow_neg[
                p, t, H2_Pipe_Map[(H2_Pipe_Map[!, :Zone].==z).&(H2_Pipe_Map[!, :pipe_no].==p), :,][!,:d][1]
            ] * inputs["pComp_MWh_per_tonne_Pipe"][p] for p in H2_Pipe_Map[H2_Pipe_Map[!, :Zone].==z, :][!, :pipe_no]
        )
    )
end

EP[:ePowerBalance] += -ePowerBalanceH2PipeCompression
EP[:eH2NetpowerConsumptionByAll] += ePowerBalanceH2PipeCompression

168×3 Matrix{AffExpr}:
 vP2G[5,1] + vP2G[8,1] + vP2G[1,1] + 2 vH2_CHARGE_STOR[11,1] + 1.9844672 vH2PipeFlow_neg[1,1,1] + 2.572308 vH2PipeFlow_neg[2,1,1] + 0.67 vH2Ncharged[1,1,1] + 47.3 vH2Ncharged[1,2,1] + 18000 vH2Ncharged[1,3,1] + 1.9844672 vH2PipeFlow_neg[1,1,1] + 2.572308 vH2PipeFlow_neg[2,1,1]                        …  vP2G[7,1] + vP2G[10,1] + vP2G[3,1] + 2 vH2_CHARGE_STOR[13,1] + 2.572308 vH2PipeFlow_neg[2,1,-1] + 2.572308 vH2PipeFlow_neg[3,1,-1] + 0.67 vH2Ncharged[3,1,1] + 47.3 vH2Ncharged[3,2,1] + 18000 vH2Ncharged[3,3,1] + 2.572308 vH2PipeFlow_neg[2,1,-1] + 2.572308 vH2PipeFlow_neg[3,1,-1]
 vP2G[5,2] + vP2G[8,2] + vP2G[1,2] + 2 vH2_CHARGE_STOR[11,2] + 1.9844672 vH2PipeFlow_neg[1,2,1] + 2.572308 vH2PipeFlow_neg[2,2,1] + 0.67 vH2Ncharged[1,1,2] + 47.3 vH2Ncharged[1,2,2] + 18000 vH2Ncharged[1,3,2] + 1.9844672 vH2PipeFlow_neg[1,2,1] + 2.572308 vH2PipeFlow_neg[2,2,1]                           vP2G[7,2] + vP2G[10,2] + vP2G[3,2] + 2 vH2_CHARGE_STOR[13,2] + 2.572308 vH2PipeFlow_neg[2

In [111]:

# H2 balance - net flows of H2 from between z and zz via pipeline p over time period t

unregister(EP, :ePipeZoneDemand)
@expression(
    EP,
    ePipeZoneDemand[t = 1:T, z = 1:Z],
    sum(
        eH2PipeFlow_net[p, t, H2_Pipe_Map[(H2_Pipe_Map[!, :Zone].==z).&(H2_Pipe_Map[!, :pipe_no].==p), :][!,:d][1]] 
        for p in H2_Pipe_Map[H2_Pipe_Map[!, :Zone].==z, :][!, :pipe_no]
    )
)

168×3 Matrix{AffExpr}:
 vH2PipeFlow_pos[1,1,1] - vH2PipeFlow_neg[1,1,1] + vH2PipeFlow_pos[2,1,1] - vH2PipeFlow_neg[2,1,1]          …  vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1]
 vH2PipeFlow_pos[1,2,1] - vH2PipeFlow_neg[1,2,1] + vH2PipeFlow_pos[2,2,1] - vH2PipeFlow_neg[2,2,1]             vH2PipeFlow_pos[2,2,-1] - vH2PipeFlow_neg[2,2,-1] + vH2PipeFlow_pos[3,2,-1] - vH2PipeFlow_neg[3,2,-1]
 vH2PipeFlow_pos[1,3,1] - vH2PipeFlow_neg[1,3,1] + vH2PipeFlow_pos[2,3,1] - vH2PipeFlow_neg[2,3,1]             vH2PipeFlow_pos[2,3,-1] - vH2PipeFlow_neg[2,3,-1] + vH2PipeFlow_pos[3,3,-1] - vH2PipeFlow_neg[3,3,-1]
 vH2PipeFlow_pos[1,4,1] - vH2PipeFlow_neg[1,4,1] + vH2PipeFlow_pos[2,4,1] - vH2PipeFlow_neg[2,4,1]             vH2PipeFlow_pos[2,4,-1] - vH2PipeFlow_neg[2,4,-1] + vH2PipeFlow_pos[3,4,-1] - vH2PipeFlow_neg[3,4,-1]
 vH2PipeFlow_pos[1,5,1] - vH2PipeFlow_neg[1,5,1] + vH2PipeFlow_pos[2,5,1] - vH2PipeFlow_neg[2,5,1]             vH2PipeFlow_po

In [112]:
 EP[:eH2Balance] += ePipeZoneDemand


168×3 Matrix{AffExpr}:
 vH2Gen[5,1] + vH2Gen[8,1] + vH2Gen[1,1] + vH2NSE[1,1,1] + vH2Gen[11,1] - vH2_CHARGE_STOR[11,1] + vH2PipeFlow_pos[1,1,1] - vH2PipeFlow_neg[1,1,1] + vH2PipeFlow_pos[2,1,1] - vH2PipeFlow_neg[2,1,1] + vH2TruckFlow[1,1,1] - vH2G2P[1,1] + vH2PipeFlow_pos[1,1,1] - vH2PipeFlow_neg[1,1,1] + vH2PipeFlow_pos[2,1,1] - vH2PipeFlow_neg[2,1,1]                                  …  vH2Gen[7,1] + vH2Gen[10,1] + vH2Gen[3,1] + vH2NSE[1,1,3] + vH2Gen[13,1] - vH2_CHARGE_STOR[13,1] + vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1] + vH2TruckFlow[3,1,1] - vH2G2P[3,1] + vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1]
 vH2Gen[5,2] + vH2Gen[8,2] + vH2Gen[1,2] + vH2NSE[1,2,1] + vH2Gen[11,2] - vH2_CHARGE_STOR[11,2] + vH2PipeFlow_pos[1,2,1] - vH2PipeFlow_neg[1,2,1] + vH2PipeFlow_pos[2,2,1] - vH2PipeFlow_neg[2,2,1] + vH2TruckFlow[1,1,2] - vH2G2P[1,2] + vH2PipeFlow_pos[1,2,1] - vH2PipeFlo

In [113]:
EP[:eHTransmissionByZone] += ePipeZoneDemand


168×3 Matrix{AffExpr}:
 vH2PipeFlow_pos[1,1,1] - vH2PipeFlow_neg[1,1,1] + vH2PipeFlow_pos[2,1,1] - vH2PipeFlow_neg[2,1,1] + vH2PipeFlow_pos[1,1,1] - vH2PipeFlow_neg[1,1,1] + vH2PipeFlow_pos[2,1,1] - vH2PipeFlow_neg[2,1,1]                  …  vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1] + vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1]
 vH2PipeFlow_pos[1,2,1] - vH2PipeFlow_neg[1,2,1] + vH2PipeFlow_pos[2,2,1] - vH2PipeFlow_neg[2,2,1] + vH2PipeFlow_pos[1,2,1] - vH2PipeFlow_neg[1,2,1] + vH2PipeFlow_pos[2,2,1] - vH2PipeFlow_neg[2,2,1]                     vH2PipeFlow_pos[2,2,-1] - vH2PipeFlow_neg[2,2,-1] + vH2PipeFlow_pos[3,2,-1] - vH2PipeFlow_neg[3,2,-1] + vH2PipeFlow_pos[2,2,-1] - vH2PipeFlow_neg[2,2,-1] + vH2PipeFlow_pos[3,2,-1] - vH2PipeFlow_neg[3,2,-1]
 vH2PipeFlow_pos[1,3,1] - vH2PipeFlow_neg[1,3,1] + vH2PipeFlow_pos[2,3,1] - vH2PipeFlow_neg[2,3,1] + vH2PipeFlow_pos[1,3,1] - v

In [114]:
# Constraints
if setup["H2PipeInteger"] == 1
    for p = 1:H2_P
        set_integer.(vH2NPipe[p])
    end
end

# Modeling expansion of the pipleline network
if setup["H2NetworkExpansion"] == 1
    # If network expansion allowed Total no. of Pipes >= Existing no. of Pipe 
    @constraints(EP, begin
        [p in 1:H2_P], EP[:eH2NPipeNew][p] >= 0
    end)
else
    # If network expansion is not alllowed Total no. of Pipes == Existing no. of Pipe 
    @constraints(EP, begin
        [p in 1:H2_P], EP[:eH2NPipeNew][p] == 0
    end)
end

(ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}[vH2NPipe[1] ≥ 0, vH2NPipe[2] ≥ 0, vH2NPipe[3] ≥ 0],)

In [115]:
# Constraint maximum pipe flow
@constraints(
    EP,
    begin
        [p in 1:H2_P, t = 1:T, d in [-1, 1]],
        EP[:eH2PipeFlow_net][p, t, d] <=
        EP[:vH2NPipe][p] * inputs["pH2_Pipe_Max_Flow"][p]
        [p in 1:H2_P, t = 1:T, d in [-1, 1]],
        -EP[:eH2PipeFlow_net][p, t, d] <=
        EP[:vH2NPipe][p] * inputs["pH2_Pipe_Max_Flow"][p]
    end
)

(3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, Base.OneTo(168)
    Dimension 3, [-1, 1]
And data, a 3×168×2 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, -1] =
 -38.8 vH2NPipe[1] + vH2PipeFlow_pos[1,1,-1] - vH2PipeFlow_neg[1,1,-1] ≤ 0  …  -38.8 vH2NPipe[1] + vH2PipeFlow_pos[1,168,-1] - vH2PipeFlow_neg[1,168,-1] ≤ 0
 -38.8 vH2NPipe[2] + vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,-1] ≤ 0     -38.8 vH2NPipe[2] + vH2PipeFlow_pos[2,168,-1] - vH2PipeFlow_neg[2,168,-1] ≤ 0
 -38.8 vH2NPipe[3] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,-1] ≤ 0     -38.8 vH2NPipe[3] + vH2PipeFlow_pos[3,168,-1] - vH2PipeFlow_neg[3,168,-1] ≤ 0

[:, :, 1] =
 -38.8 vH2NPipe[1] + vH2P

In [116]:
# Minimum and maximum pipe level constraint
@constraints(
    EP,
    begin
        [p in 1:H2_P, t = 1:T],
        vH2PipeLevel[p, t] >= inputs["pH2_Pipe_Min_Cap"][p] * vH2NPipe[p]
        [p in 1:H2_P, t = 1:T],
        inputs["pH2_Pipe_Max_Cap"][p] * vH2NPipe[p] >= vH2PipeLevel[p, t]
    end
)

(ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}[-14.520891200000001 vH2NPipe[1] + vH2PipeLevel[1,1] ≥ 0 -14.520891200000001 vH2NPipe[1] + vH2PipeLevel[1,2] ≥ 0 … -14.520891200000001 vH2NPipe[1] + vH2PipeLevel[1,167] ≥ 0 -14.520891200000001 vH2NPipe[1] + vH2PipeLevel[1,168] ≥ 0; -23.191543 vH2NPipe[2] + vH2PipeLevel[2,1] ≥ 0 -23.191543 vH2NPipe[2] + vH2PipeLevel[2,2] ≥ 0 … -23.191543 vH2NPipe[2] + vH2PipeLevel[2,167] ≥ 0 -23.191543 vH2NPipe[2] + vH2PipeLevel[2,168] ≥ 0; -23.191543 vH2NPipe[3] + vH2PipeLevel[3,1] ≥ 0 -23.191543 vH2NPipe[3] + vH2PipeLevel[3,2] ≥ 0 … -23.191543 vH2NPipe[3] + vH2PipeLevel[3,167] ≥ 0 -23.191543 vH2NPipe[3] + vH2PipeLevel[3,168] ≥ 0], ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}[145.208912 vH2NPipe[1] - vH2PipeLevel[1,1] ≥ 0 145.208912 vH2NPipe[

In [117]:
# Pipeline storage level change
@constraints(
    EP,
    begin
        [p in 1:H2_P, t in START_SUBPERIODS],
        vH2PipeLevel[p, t] ==
        vH2PipeLevel[p, t+hours_per_subperiod-1] - eH2PipeFlow_net[p, t, -1] -
        eH2PipeFlow_net[p, t, 1]
    end
)


(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, 1:168:1
And data, a 3×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 vH2PipeLevel[1,1] - vH2PipeLevel[1,168] + vH2PipeFlow_pos[1,1,1] + vH2PipeFlow_pos[1,1,-1] - vH2PipeFlow_neg[1,1,1] - vH2PipeFlow_neg[1,1,-1] = 0
 vH2PipeLevel[2,1] - vH2PipeLevel[2,168] + vH2PipeFlow_pos[2,1,1] + vH2PipeFlow_pos[2,1,-1] - vH2PipeFlow_neg[2,1,1] - vH2PipeFlow_neg[2,1,-1] = 0
 vH2PipeLevel[3,1] - vH2PipeLevel[3,168] + vH2PipeFlow_pos[3,1,1] + vH2PipeFlow_pos[3,1,-1] - vH2PipeFlow_neg[3,1,1] - vH2PipeFlow_neg[3,1,-1] = 0,)

In [118]:

@constraints(
    EP,
    begin
        [p in 1:H2_P, t in INTERIOR_SUBPERIODS],
        vH2PipeLevel[p, t] ==
        vH2PipeLevel[p, t-1] - eH2PipeFlow_net[p, t, -1] - eH2PipeFlow_net[p, t, 1]
    end
)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, Base.OneTo(3)
    Dimension 2, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  159, 160, 161, 162, 163, 164, 165, 166, 167, 168]
And data, a 3×167 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -vH2PipeLevel[1,1] + vH2PipeLevel[1,2] + vH2PipeFlow_pos[1,2,1] + vH2PipeFlow_pos[1,2,-1] - vH2PipeFlow_neg[1,2,1] - vH2PipeFlow_neg[1,2,-1] = 0  …  -vH2PipeLevel[1,167] + vH2PipeLevel[1,168] + vH2PipeFlow_pos[1,168,1] + vH2PipeFlow_pos[1,168,-1] - vH2PipeFlow_neg[1,168,1] - vH2PipeFlow_neg[1,168,-1] = 0
 -vH2PipeLevel[2,1] + vH2PipeLevel[2,2] + vH2PipeFlow_pos[2,2,1] + vH2PipeFlow_pos[2,2,-1] - vH2PipeFlow_neg[2,2,1] - vH2PipeFlow_neg[2,2,-1] = 0     -vH2PipeLevel[2,167] + vH2

In [119]:
@constraints(EP, begin
    [p in 1:H2_P], vH2NPipe[p] <= inputs["pH2_Pipe_No_Max"][p]
end)

(ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}[vH2NPipe[1] ≤ 20, vH2NPipe[2] ≤ 20, vH2NPipe[3] ≤ 20],)